In [9]:
import keras
from keras.models import Sequential
from keras.utils import np_utils
from keras.preprocessing.image import ImageDataGenerator
from keras.layers import Dense, Activation, Flatten, Dropout, BatchNormalization
from keras.layers import Conv2D, MaxPooling2D,AveragePooling2D,GlobalAveragePooling2D
from keras.datasets import cifar10
from keras import regularizers, optimizers
import numpy as np
from keras.layers import Add
from keras.layers import Input
from keras.models import Model
from keras.layers import Flatten

In [35]:
from keras import backend as K

In [15]:
def initial_conv(Input, filters, stride = 1,kernel_size = 7):
    x = Conv2D(filters, kernel_size=(kernel_size,kernel_size), strides = (stride,stride), padding = "same")(Input)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    return x

In [20]:
def expand_conv_basic_block(Input, filters, stride=1, dropout = 0.0):
    Init = Input
    x = Conv2D(filters,kernel_size=(3,3),strides = (stride,stride),padding = "same")(Input)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    if(dropout > 0.0):
        x = Dropout(dropout)(x)
    x = Conv2D(filters,kernel_size=(3,3),strides = (1,1),padding = "same")(x)
    x = BatchNormalization()(x)
    skip_conv = Conv2D(filters, kernel_size = (1,1),strides = (stride,stride),padding = "same")(Input)
    skip = BatchNormalization()(skip_conv)
    x = Add()([x,skip])
    return x

In [21]:
def normal_conv_basic_block(Input, filters, stride = 1, dropout = 0.0):
    x = Conv2D(filters,kernel_size=(3,3),strides = (stride,stride),padding = "same")(Input)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    if(dropout > 0.0):
        x = Dropout(dropout)(x)
    x = Conv2D(filters,kernel_size=(3,3),strides = (stride,stride),padding = "same")(x)
    x = BatchNormalization()(x)
    x = Add()([x,Input])
    return x

In [51]:
def expand_conv_bottleneck_block(Input,filters,stride=1,dropout = 0.0):
    x = Conv2D(filters,kernel_size=(1,1),strides = (stride,stride),padding = "same")(Input)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    #if(dropout > 0.0):
     #   x = Dropout(dropout)(x)
    x = Conv2D(filters,kernel_size=(3,3),strides = (1,1),padding = "same")(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    #if(Dropout > 0.0):
     #   x = Dropout(dropout)(x)
    x = Conv2D(filters*4,kernel_size=(1,1),strides = (1,1),padding = "same")(x)
    x = BatchNormalization()(x)
    skip_conv = Conv2D(filters*4,kernel_size = (1,1), strides = (stride, stride),padding = "same")(Input)
    skip = BatchNormalization()(skip_conv)
    x = Add()([x,skip])
    return x

In [52]:
def normal_conv_bottleneck_block(Input, filters, stride = 1, dropout = 0.0):
    x = Conv2D(filters,kernel_size=(1,1),strides = (stride,stride),padding = "same")(Input)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    #if(dropout > 0.0):
     #   x = Dropout(dropout)(x)
    x = Conv2D(filters,kernel_size=(3,3),strides = (stride,stride),padding = "same")(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
   # if(Dropout > 0.0):
    #    x = Dropout(dropout)(x)
    x = Conv2D(filters*4,kernel_size=(1,1),strides = (stride,stride),padding = "same")(x)
    x = BatchNormalization()(x)
    x = Add()([x,Input])
    return x

In [53]:
def build_basic_resnet(h, w, no_of_outputs, r1,r2,r3,r4, first_conv_stride = 2, first_max_pool = True,first_conv_kernel_size = 7):
    
    inputs = Input(shape = (h,w,3), name = "image_input")
    
    x = initial_conv(inputs,64,first_conv_stride,first_conv_kernel_size)
    
    if(first_max_pool):
        x = MaxPooling2D(pool_size=(2,2))(x)
    
    x = expand_conv_basic_block(x,64,1)
    x = Activation('relu')(x)
    for i in range(r1-1):
        x = normal_conv_basic_block(x,64)
        x = Activation('relu')(x)
    x = expand_conv_basic_block(x,128,2)
    x = Activation('relu')(x)
    for i in range(r2-1):
        x = normal_conv_basic_block(x,128)
        x = Activation('relu')(x)
    x = expand_conv_basic_block(x,256,2)
    x = Activation('relu')(x)
    for i in range(r3-1):
        x = normal_conv_basic_block(x,256)
        x = Activation('relu')(x)
            
    x = expand_conv_basic_block(x,512,2)
    x = Activation('relu')(x)
    for i in range(r4-1):
        x = normal_conv_basic_block(x,512)
        x = Activation('relu')(x)
    
    shape = K.int_shape(x)
    x = AveragePooling2D(pool_size=(shape[1], shape[2]),
                                 strides=(1, 1))(x)
   # x = GlobalAveragePooling2D()(x)
    x = Flatten()(x)
    x = Dense(no_of_outputs,activation='softmax')(x)
    
    model = Model(inputs = inputs, outputs = x)
    return model
    
    

In [40]:
model = build_basic_resnet(32,32,10,2,2,2,2,1,False,7)

In [41]:
model.summary()

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
image_input (InputLayer)         (None, 32, 32, 3)     0                                            
____________________________________________________________________________________________________
conv2d_158 (Conv2D)              (None, 32, 32, 64)    9472        image_input[0][0]                
____________________________________________________________________________________________________
batch_normalization_158 (BatchNo (None, 32, 32, 64)    256         conv2d_158[0][0]                 
____________________________________________________________________________________________________
activation_127 (Activation)      (None, 32, 32, 64)    0           batch_normalization_158[0][0]    
___________________________________________________________________________________________

In [42]:
from keras.utils import plot_model 

In [43]:
plot_model(model,"Resnet.png",show_shapes=True)

In [54]:
def build_bottleneck_resnet(h, w, no_of_outputs, r1,r2,r3,r4, first_conv_stride = 2, first_max_pool = True,first_conv_kernel_size = 7):
    
    inputs = Input(shape = (h,w,3), name = "image_input")
    
    x = initial_conv(inputs,64,first_conv_stride,first_conv_kernel_size)
    
    if(first_max_pool):
        x = MaxPooling2D(pool_size=(2,2))(x)
    
    x = expand_conv_bottleneck_block(x,64,1)
    x = Activation('relu')(x)
    for i in range(r1-1):
        x = normal_conv_bottleneck_block(x,64)
        x = Activation('relu')(x)
    x = expand_conv_bottleneck_block(x,128,2)
    x = Activation('relu')(x)
    for i in range(r2-1):
        x = normal_conv_bottleneck_block(x,128)
        x = Activation('relu')(x)
    x = expand_conv_bottleneck_block(x,256,2)
    x = Activation('relu')(x)
    for i in range(r3-1):
        x = normal_conv_bottleneck_block(x,256)
        x = Activation('relu')(x)
            
    x = expand_conv_bottleneck_block(x,512,2)
    x = Activation('relu')(x)
    for i in range(r4-1):
        x = normal_conv_bottleneck_block(x,512)
        x = Activation('relu')(x)
    
    shape = K.int_shape(x)
    x = AveragePooling2D(pool_size=(shape[1], shape[2]),
                                 strides=(1, 1))(x)
   # x = GlobalAveragePooling2D()(x)
    x = Flatten()(x)
    x = Dense(no_of_outputs,activation='softmax')(x)
    
    model = Model(inputs = inputs, outputs = x)
    return model
    
    

In [55]:
model = build_bottleneck_resnet(32,32,10,3,4,6,3,1,False,7)

In [56]:
plot_model(model,"Resnet_bottleneck.png",show_shapes=True)

In [50]:
model.summary()

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
image_input (InputLayer)         (None, 32, 32, 3)     0                                            
____________________________________________________________________________________________________
conv2d_208 (Conv2D)              (None, 32, 32, 64)    9472        image_input[0][0]                
____________________________________________________________________________________________________
batch_normalization_208 (BatchNo (None, 32, 32, 64)    256         conv2d_208[0][0]                 
____________________________________________________________________________________________________
activation_169 (Activation)      (None, 32, 32, 64)    0           batch_normalization_208[0][0]    
___________________________________________________________________________________________